In [37]:
pip install scipy

You should consider upgrading via the '/Users/anaistang/.virtualenvs/web/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [87]:
import requests 
import pandas as pd 
import requests 
import math 
from scipy import stats
import xlsxwriter

In [40]:
from starter.secrets import IEX_CLOUD_API_TOKEN
stocks = pd.read_csv('starter/sp_500_stocks.csv')


## Getting one-year price returns for each stock in the universe

In [5]:
symbol = 'aapl'
url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
r = requests.get(url)
data = r.json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2259997339949,
 'week52high': 151.41,
 'week52low': 88.3,
 'week52highSplitAdjustOnly': 150.4,
 'week52lowSplitAdjustOnly': 89,
 'week52change': 0.5083248069430318,
 'sharesOutstanding': 17322928475,
 'float': 0,
 'avg10Volume': 83648384,
 'avg30Volume': 76372161,
 'day200MovingAvg': 133.1,
 'day50MovingAvg': 130.52,
 'employees': 149515,
 'ttmEPS': 4.64,
 'ttmDividendRate': 0.854889345932639,
 'dividendYield': 0.006437005696199516,
 'nextDividendDate': '',
 'exDividendDate': '2021-04-30',
 'nextEarningsDate': '2021-07-15',
 'peRatio': 29.809175981594365,
 'beta': 1.55591893680141,
 'maxChangePercent': 49.61249743608207,
 'year5ChangePercent': 5.065695169097023,
 'year2ChangePercent': 1.73752821140607,
 'year1ChangePercent': 0.4977837070870686,
 'ytdChangePercent': -0.013771508361123437,
 'month6ChangePercent': 0.03467239533035248,
 'month3ChangePercent': 0.08695595676480292,
 'month1ChangePercent': 0.04556023561983153,
 'day30ChangePercent': 

## Parsing API call


In [6]:
change_percent = data['year1ChangePercent']
change_percent

0.4977837070870686

In [7]:
def chunks(lst, n):
    """
    Splitting list into evenly sized chunks
    """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]



In [8]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [9]:
columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns=columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     print(data['A'])
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
        pd.Series(
            [
                symbol, 
                data[symbol]['stats']['week52high'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
            ],
            index=columns),
            ignore_index=True
            )
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,150.96,0.671627,N/A
1,AAL,26.21,0.364777,N/A
2,AAP,212.95,0.327811,N/A
3,AAPL,145.14,0.502128,N/A
4,ABBV,119.27,0.242266,N/A
...,...,...,...,...
95,CINF,123.90,0.851633,N/A
96,CL,89.13,0.131360,N/A
97,CLX,241.00,-0.177825,N/A
98,CMA,81.31,0.787049,N/A


## Removing Low-Momentum Stocks
- We're identifying 50 highest-momentum stocks in the S&P 500
- We need to remove all the stocks in our DataFrame that fall below this momentum threshold.
- Sort the Dataframe by the stocks one-year price return, drop all stocks outside the top 50.

In [143]:
final_df.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_df = final_df[:50]
final_df.reset_index(inplace=True, drop=True) # sets first row as first index 
final_df 

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,24,ALGN,668.20,1.337336,29931
1,29,AMAT,149.63,1.216316,133663
2,78,CARR,48.92,1.181510,408830
3,23,ALB,192.18,1.041956,104069
4,45,APTV,166.47,0.948072,120141
5,16,AES,29.52,0.937285,677506
6,95,CINF,123.90,0.851633,161420
7,82,CBRE,91.92,0.823169,217580
8,51,AVY,231.12,0.795635,86535
9,98,CMA,81.31,0.787049,245972


In [11]:
## Calculating the number of shares to buy 

In [135]:
def portfolio_size():
    global portfolio_size 
    portfolio_size = input('Enter the size of your portfolio:\n')
    
    try:
        float(portfolio_size)
    except:
        print('This is not a number \n Please try again:')
        portfolio_size = input('Enter the size of your portfolio')
        
portfolio_size()
# print(portfolio_size)
        

Enter the size of your portfolio:
10000000


In [64]:
position_size = float(portfolio_size)/len(final_df.index)
# print(position_size)
for i in range(0, len(final_df)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Price'])
final_df

/Users/anaistang/.virtualenvs/web/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,24,ALGN,668.20,1.337336,29931
1,29,AMAT,149.63,1.216316,133663
2,78,CARR,48.92,1.181510,408830
3,23,ALB,192.18,1.041956,104069
4,45,APTV,166.47,0.948072,120141
5,16,AES,29.52,0.937285,677506
6,95,CINF,123.90,0.851633,161420
7,82,CBRE,91.92,0.823169,217580
8,51,AVY,231.12,0.795635,86535
9,98,CMA,81.31,0.787049,245972


## Building a realistic momentum strategy 
- High-quality momentum stocks show 'slow and steady' outperformance over long periods of time 
- Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards

High-quality momentum stocks are preferred because low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future (such as FDA approval for a biotech company)

To identify high-quality momentum, building the following strategies will help select stocks from highest percentiles of
1. 1-month price returns 
2. 3-month price returns 
3. 6-month price returns 
4. 1-year price returns 


In [111]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-year price return',
    'One-year return percentile',
    'Six-month price return',
    'Six-month return percentile',
    'Three-month price return',
    'Three-month return percentile',
    'One-month price return',
    'One-month return percentile',
    'HQM Score'
]

hqm_df = pd.DataFrame(columns=hqm_columns)
convert_none = lambda x: 0 if x is None else x

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
#         print(data)
        hqm_df = hqm_df.append(
            
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
                
            ],
            index = hqm_columns),
            ignore_index=True
        )

hqm_df.fillna(value=0, inplace=True)
hqm_df
    

,Ticker,Price,Number of Shares to Buy,One-year price return,One-year return percentile,Six-month price return,Six-month return percentile,Three-month price return,Three-month return percentile,One-month price return,One-month return percentile,HQM Score
0,A,147.07,N/A,0.668291,N/A,0.224416,N/A,0.193532,N/A,0.128079,N/A,N/A
1,AAL,22.88,N/A,0.353134,N/A,0.361852,N/A,-0.098212,N/A,-0.055002,N/A,N/A
2,AAP,202.36,N/A,0.335014,N/A,0.217950,N/A,0.059595,N/A,-0.029322,N/A,N/A
3,AAPL,133.00,N/A,0.507163,N/A,0.033856,N/A,0.087393,N/A,0.047014,N/A,N/A
4,ABBV,113.84,N/A,0.237403,N/A,0.111438,N/A,0.104243,N/A,-0.036512,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.56,N/A,0.300912,N/A,0.056209,N/A,0.063136,N/A,-0.036590,N/A,N/A
501,ZBH,158.63,N/A,0.254074,N/A,0.047384,N/A,-0.000885,N/A,-0.049978,N/A,N/A
502,ZBRA,503.91,N/A,0.924404,N/A,0.322884,N/A,0.071169,N/A,0.033337,N/A,N/A
503,ZION,52.80,N/A,0.436420,N/A,0.243938,N/A,-0.118640,N/A,-0.138096,N/A,N/A


## Calculating Momentum Percentiles 

- Calculate momentum percentile scores for every stock in the universe
- We need to calculate percentile scores for the following metrics for every stock:

    - `One-Year price return`
    - `Six month price return`
    - `Three-month price return`
    - `One-month price return`


In [114]:
time_periods = [
                'One-year',
                'Six-month',
                'Three-month',
                'One-month'
                ]

for row in hqm_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} price return'
        percentile_col = f'{time_period} return percentile'            
        score = stats.percentileofscore(hqm_df[change_col],
                                        hqm_df.loc[row, change_col])

        hqm_df.loc[row, percentile_col] = math.floor(score)
hqm_df.replace([None], 0, inplace=True)
hqm_df



                                                                                                                                                                  


,Ticker,Price,Number of Shares to Buy,One-year price return,One-year return percentile,Six-month price return,Six-month return percentile,Three-month price return,Three-month return percentile,One-month price return,One-month return percentile,HQM Score
0,A,147.07,N/A,0.668291,80.0,0.224416,67.0,0.193532,91.0,0.128079,96.0,N/A
1,AAL,22.88,N/A,0.353134,46.0,0.361852,88.0,-0.098212,7.0,-0.055002,29.0,N/A
2,AAP,202.36,N/A,0.335014,44.0,0.217950,66.0,0.059595,58.0,-0.029322,44.0,N/A
3,AAPL,133.00,N/A,0.507163,65.0,0.033856,20.0,0.087393,69.0,0.047014,84.0,N/A
4,ABBV,113.84,N/A,0.237403,30.0,0.111438,40.0,0.104243,73.0,-0.036512,40.0,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.56,N/A,0.300912,39.0,0.056209,26.0,0.063136,59.0,-0.036590,40.0,N/A
501,ZBH,158.63,N/A,0.254074,32.0,0.047384,23.0,-0.000885,31.0,-0.049978,32.0,N/A
502,ZBRA,503.91,N/A,0.924404,91.0,0.322884,85.0,0.071169,61.0,0.033337,78.0,N/A
503,ZION,52.80,N/A,0.436420,56.0,0.243938,72.0,-0.118640,3.0,-0.138096,2.0,N/A


## Calculating the HQM Score
- To calculate `HQM Score` (high quality momentum), we need to calculate the arithmetic mean of the 4 momentum percentile scores that we calculated using `mean` function in the `statistics` module

In [119]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} return percentile'])
    # sum(80, 67, 91, 96)/ 4
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-year price return,One-year return percentile,Six-month price return,Six-month return percentile,Three-month price return,Three-month return percentile,One-month price return,One-month return percentile,HQM Score
0,A,147.07,N/A,0.668291,80.0,0.224416,67.0,0.193532,91.0,0.128079,96.0,83.5
1,AAL,22.88,N/A,0.353134,46.0,0.361852,88.0,-0.098212,7.0,-0.055002,29.0,42.5
2,AAP,202.36,N/A,0.335014,44.0,0.217950,66.0,0.059595,58.0,-0.029322,44.0,53.0
3,AAPL,133.00,N/A,0.507163,65.0,0.033856,20.0,0.087393,69.0,0.047014,84.0,59.5
4,ABBV,113.84,N/A,0.237403,30.0,0.111438,40.0,0.104243,73.0,-0.036512,40.0,45.75
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.56,N/A,0.300912,39.0,0.056209,26.0,0.063136,59.0,-0.036590,40.0,41.0
501,ZBH,158.63,N/A,0.254074,32.0,0.047384,23.0,-0.000885,31.0,-0.049978,32.0,29.5
502,ZBRA,503.91,N/A,0.924404,91.0,0.322884,85.0,0.071169,61.0,0.033337,78.0,78.75
503,ZION,52.80,N/A,0.436420,56.0,0.243938,72.0,-0.118640,3.0,-0.138096,2.0,33.25


## Selecting the 5 best Momentum Stocks
- Identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries

In [142]:
hqm_df.sort_values('HQM Score', ascending=False, inplace=True)
hqm_df = hqm_df[:50]
hqm_df

,index,Ticker,Price,Number of Shares to Buy,One-year price return,One-year return percentile,Six-month price return,Six-month return percentile,Three-month price return,Three-month return percentile,One-month price return,One-month return percentile,HQM Score
0,345,NVDA,765.79,261,1.066700,95.0,0.417526,92.0,0.469988,99.0,0.339331,99.0,96.25
1,174,F,14.78,13531,1.310886,98.0,0.636046,98.0,0.164583,86.0,0.204859,99.0,95.25
2,195,FTNT,234.27,853,0.730511,84.0,0.619019,98.0,0.359239,99.0,0.150396,97.0,94.5
3,436,TGT,239.49,835,0.993237,94.0,0.369812,89.0,0.292521,98.0,0.117707,95.0,94.0
4,29,AMAT,135.71,1473,1.247535,97.0,0.550932,97.0,0.154003,85.0,0.079898,91.0,92.5
5,477,WAT,359.01,557,0.801039,87.0,0.370177,89.0,0.284081,97.0,0.128199,96.0,92.25
6,317,MRO,13.02,15360,0.992406,93.0,0.892027,99.0,0.189173,89.0,0.056122,86.0,91.75
7,204,GOOG,2593.88,77,0.773891,86.0,0.451814,94.0,0.244824,94.0,0.092115,93.0,91.75
8,175,FANG,87.03,2298,0.908533,90.0,0.905813,100.0,0.153312,84.0,0.079249,90.0,91.0
9,249,IRM,45.36,4409,0.771246,86.0,0.569055,97.0,0.262345,95.0,0.053716,85.0,90.75


In [145]:
position_size = float(portfolio_size)/len(hqm_df.index)
# position_size
for i in hqm_df.index:
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[i, 'Price'])
hqm_df.reset_index(drop=True)
hqm_df

,index,Ticker,Price,Number of Shares to Buy,One-year price return,One-year return percentile,Six-month price return,Six-month return percentile,Three-month price return,Three-month return percentile,One-month price return,One-month return percentile,HQM Score
0,345,NVDA,765.79,261,1.066700,95.0,0.417526,92.0,0.469988,99.0,0.339331,99.0,96.25
1,174,F,14.78,13531,1.310886,98.0,0.636046,98.0,0.164583,86.0,0.204859,99.0,95.25
2,195,FTNT,234.27,853,0.730511,84.0,0.619019,98.0,0.359239,99.0,0.150396,97.0,94.5
3,436,TGT,239.49,835,0.993237,94.0,0.369812,89.0,0.292521,98.0,0.117707,95.0,94.0
4,29,AMAT,135.71,1473,1.247535,97.0,0.550932,97.0,0.154003,85.0,0.079898,91.0,92.5
5,477,WAT,359.01,557,0.801039,87.0,0.370177,89.0,0.284081,97.0,0.128199,96.0,92.25
6,317,MRO,13.02,15360,0.992406,93.0,0.892027,99.0,0.189173,89.0,0.056122,86.0,91.75
7,204,GOOG,2593.88,77,0.773891,86.0,0.451814,94.0,0.244824,94.0,0.092115,93.0,91.75
8,175,FANG,87.03,2298,0.908533,90.0,0.905813,100.0,0.153312,84.0,0.079249,90.0,91.0
9,249,IRM,45.36,4409,0.771246,86.0,0.569055,97.0,0.262345,95.0,0.053716,85.0,90.75


## Formatting excel output
- Using XlsxWriter library for Python to create formatted Excel files


In [147]:
writer = pd.ExcelWriter('momentum_strat.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name='Momentum Strategy', index=False)

,index,Ticker,Price,Number of Shares to Buy,One-year price return,One-year return percentile,Six-month price return,Six-month return percentile,Three-month price return,Three-month return percentile,One-month price return,One-month return percentile,HQM Score
0,345,NVDA,765.79,261,1.066700,95.0,0.417526,92.0,0.469988,99.0,0.339331,99.0,96.25
1,174,F,14.78,13531,1.310886,98.0,0.636046,98.0,0.164583,86.0,0.204859,99.0,95.25
2,195,FTNT,234.27,853,0.730511,84.0,0.619019,98.0,0.359239,99.0,0.150396,97.0,94.5
3,436,TGT,239.49,835,0.993237,94.0,0.369812,89.0,0.292521,98.0,0.117707,95.0,94.0
4,29,AMAT,135.71,1473,1.247535,97.0,0.550932,97.0,0.154003,85.0,0.079898,91.0,92.5
5,477,WAT,359.01,557,0.801039,87.0,0.370177,89.0,0.284081,97.0,0.128199,96.0,92.25
6,317,MRO,13.02,15360,0.992406,93.0,0.892027,99.0,0.189173,89.0,0.056122,86.0,91.75
7,204,GOOG,2593.88,77,0.773891,86.0,0.451814,94.0,0.244824,94.0,0.092115,93.0,91.75
8,175,FANG,87.03,2298,0.908533,90.0,0.905813,100.0,0.153312,84.0,0.079249,90.0,91.0
9,249,IRM,45.36,4409,0.771246,86.0,0.569055,97.0,0.262345,95.0,0.053716,85.0,90.75


In [148]:
background_colour = '#0a0a23'
font_colour = '#ffffff'
string_template = writer.book.add_format(
{
    'font_color': font_colour,
    'bg_color': background_colour,
    'border': 1,
})

dollar_template = writer.book.add_format(
{
    'num_format': '$0.00',
    'font_color': font_colour,
    'bg_color': background_colour,
    'border': 1
})

integer_template = writer.book.add_format(
{
    'num_format': '0',
    'font_color': font_colour,
    'bg_color': background_colour,
    'border': 1
})

percent_template = writer.book.add_format({
    'num_format': '0.00',
    'font_color': font_colour,
    'bg_color': background_colour,
    'border': 1
})

In [151]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year price return', percent_template],
    'E': ['One-Year return percentile', percent_template],
    'F': ['Six-month price return', percent_template],
    'G': ['Six-month return percentile', percent_template],
    'H': ['Three-month price return', percent_template],
    'I': ['Three-month return percentile', percent_template],
    'J': ['One-month price return ', percent_template],
    'K': ['One-month return percentile', percent_template],
    'L': ['HQM Score', integer_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f"{column}:{column}", 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)
    
writer.save()
    